# Images to Point Cloud

This notebook takes all images in the selected directory, calculates the threshold and creates stacked point cloud out of black pixels

Notebook created by [Artem Konevskikh](https://aiculedssul.net/)

In [1]:
#@title Load libraries
import glob
import numpy as np
import os
import struct
import cv2
from tqdm.notebook import tqdm

In [2]:
#@title Additional functions for point cloud generation



def rotate(plane, angle):
    theta = np.radians(angle)
    # rot_x = np.array([[1, 0, 0], [0, np.cos(theta), -np.sin(theta)], [0, np.sin(theta), np.cos(theta)]])
    rot_y = np.array([[np.cos(theta), 0, np.sin(theta)], [0, 1, 0], [-np.sin(theta), 0, np.cos(theta)]])
    # rot_z = np.array([[np.cos(theta), -np.sin(theta), 0], [np.sin(theta), np.cos(theta), 0], [0, 0, 1]])
    return plane.dot(rot_y)


def img2points(img, d=0, a=0, thr=127, invert=False):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # th = cv2.adaptiveThreshold(gray, thr, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 4)
    _, th = cv2.threshold(gray, thr, 255, cv2.THRESH_BINARY)
    # if invert:
    #   th = 255 - th
    depth = 1 - th / 255
    new_depth = depth + d
    w, h = new_depth.shape

    pixel_x, pixel_y = np.meshgrid(np.linspace(0, h - 1, h),np.linspace(0, w - 1, w))
    camera_points = np.zeros((np.size(pixel_x), 3))
    camera_points[:, 0] = np.reshape(pixel_x, -1)
    camera_points[:, 1] = np.reshape(pixel_y, -1)
    camera_points[:, 2] = np.reshape(new_depth, -1)

    color_points = img.reshape(-1, 3)
    if a % 360 != 0:
        camera_points = rotate(camera_points, a)

    valid_depth_ind = np.where(depth.flatten() > 0)[0]
    camera_points = camera_points[valid_depth_ind, :]
    color_points = color_points[valid_depth_ind, :]
    color_points = color_points.astype(int)

    return camera_points, color_points


def write_pointcloud(filename, xyz_points, rgb_points=None):
    """ creates a .ply file of the generated point clouds 
    """

    assert xyz_points.shape[1] == 3, 'Input XYZ points should be Nx3 float array'
    if rgb_points is None:
        rgb_points = np.ones(xyz_points.shape).astype(np.uint8) * 255
    assert xyz_points.shape == rgb_points.shape, 'Input RGB colors should be Nx3 float array and have same size as input XYZ points'

    # Write header of .ply file
    with open(filename, 'wb') as fid:
        fid.write(bytes('ply\n', 'utf-8'))
        fid.write(bytes('format binary_little_endian 1.0\n', 'utf-8'))
        fid.write(bytes(f'element vertex {xyz_points.shape[0]}\n', 'utf-8'))
        fid.write(bytes('property float x\n', 'utf-8'))
        fid.write(bytes('property float y\n', 'utf-8'))
        fid.write(bytes('property float z\n', 'utf-8'))
        fid.write(bytes('property uchar red\n', 'utf-8'))
        fid.write(bytes('property uchar green\n', 'utf-8'))
        fid.write(bytes('property uchar blue\n', 'utf-8'))
        fid.write(bytes('end_header\n', 'utf-8'))

        # Write 3D points to .ply file
        for i in range(xyz_points.shape[0]):
            fid.write(bytearray(struct.pack("fffBBB", xyz_points[i, 0], xyz_points[i, 1], xyz_points[i, 2],
                                            rgb_points[i, 0], rgb_points[i, 1],
                                            rgb_points[i, 2])))


def sparse_image(img, level):
    new_image = np.zeros(img.shape, dtype=np.uint8)
    new_image.fill(255)
    new_image[::level,::level] = img[::level,::level]
    return new_image

In [ ]:
#@title Mount Google Drive
#@markdown Mount Google Drive to load images and to save the results.

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title Images to Point Cloud

#@markdown Take every n-th layer
skip_layer = 1 #@param {type: "integer"}
#@markdown Take every n-th point in layer
skip_points =  10#@param {type: "integer"}
#@markdown Distance between layers (if >1 distance will be bigger, if from 0 to 1 it will be scaled down)
distance = 5 #@param {type: "number"}
#@markdown Threshold level
threshold = 23 #@param {type:"slider", min:1, max:255, step:1}
#@markdown Image directory
image_dir = "/content/drive/MyDrive/workshops/uibk/vvvv/interpolated_frames" #@param {type: "string"}
#@markdown Directory to save point cloud
results_cloud = "/content/drive/MyDrive/workshops/uibk/pointclouds" #@param {type: "string"}
#@markdown Set point cloud file name
cloud_name = "vvvv-int-1-10-5--thr23" #@param {type: "string"}
#@markdown Make cube
make_cube = False #@param {type: "boolean"}
# #@markdown Invert point cloud
# invert = True #@param {type: "boolean"}


if image_dir=="":
  raise Exception("Please specify image directory")
if image_dir[-1] != "/":
    image_dir += "/"
if results_cloud=="":
  results_cloud= '/content/'
if results_cloud[-1] != "/":
    results_cloud += "/"
if not os.path.exists(results_cloud):
  os.makedirs(results_cloud)
if cloud_name=="":
  raise Exception("Please specify point cloud file name!")

img_files = sorted(glob.glob(f'{image_dir}*.png')+glob.glob(f'{image_dir}*.jpg'))
total = len(img_files)
assert total>0, 'Image directory is empty!'
print(f'{total} images found')
all_points = np.empty((0, 3))
all_colors = np.empty((0, 3))



# for i, fname in enumerate(img_files):
for i, fname in tqdm(enumerate(img_files), total=total):
    if i % skip_layer != 0:
        continue
    image = cv2.imread(fname, 1)
    if make_cube:
      image = cv2.resize(image, (total,total), interpolation = cv2.INTER_LINEAR)
    image = sparse_image(image, skip_points)
    points, colors = img2points(image, d=i*distance, a=0, thr=threshold) #, invert=invert)
    all_points = np.concatenate((all_points, points))
    all_colors = np.concatenate((all_colors, colors))


write_pointcloud(f'{results_cloud}{cloud_name}.ply', all_points, all_colors.astype(np.uint8))
print(f'Cloud saved to {results_cloud}{cloud_name}.ply')